# Cross-Validation - Breast Cancer Detection Dataset

In [29]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,cross_val_score,cross_val_predict, StratifiedKFold
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings('ignore')

seed=101

# Milestones
- Load sklearn's breast_cancer dataset
- Ways of Data Partitioning, Training & Evaluation
    - train_test_split 
    - Cross-Validation
- Applications of Cross-Validation   
    - Choose your model
    - Hyper-parameter Tuning

In [22]:
loaded_data = load_breast_cancer()

In [23]:
df = pd.DataFrame(data= loaded_data.data, columns=[loaded_data.feature_names])

In [24]:
#Framing target column in our dataframe
df['target'] = pd.Series(data=loaded_data.target, index=df.index)

In [25]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [26]:
df.shape

(569, 31)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
(mean radius,)                569 non-null float64
(mean texture,)               569 non-null float64
(mean perimeter,)             569 non-null float64
(mean area,)                  569 non-null float64
(mean smoothness,)            569 non-null float64
(mean compactness,)           569 non-null float64
(mean concavity,)             569 non-null float64
(mean concave points,)        569 non-null float64
(mean symmetry,)              569 non-null float64
(mean fractal dimension,)     569 non-null float64
(radius error,)               569 non-null float64
(texture error,)              569 non-null float64
(perimeter error,)            569 non-null float64
(area error,)                 569 non-null float64
(smoothness error,)           569 non-null float64
(compactness error,)          569 non-null float64
(concavity error,)            569 non-null float64
(concave points error,)     

## Ways of Data Partitioning, Training & Evaluation
For building the model, data is partitioned into train/test. There are various ways of doing it, each having its own limitation

### Read and Split Data into Training and Test Sets 

In [31]:
X = loaded_data.data
y = loaded_data.target

xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.2, random_state=seed)


### Build the LogisticRegression Model - Train and Test

In [32]:
lg_model = LogisticRegression()
lg_model.fit(xtrain,ytrain)
lg_model.score(xtest,ytest)

0.956140350877193

## Cross-Validation

In [33]:
# Custom function for evaluating model score
def return_score(model,X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    score = model.score(X_test, y_test)
    return score

### Implement Manually using K-Fold & Stratified K-Fold

In [46]:
model = LogisticRegression()

scores = []
#cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=False)
cv = KFold(n_splits=5, random_state=seed, shuffle=False) 
for train_index, test_index in cv.split(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
    score = return_score(model,X_train, X_test, y_train, y_test)    
    scores.append(score)

print("Accuracy score in each iteration: \n{}\n".format(scores))
print("K-Fold Score: {}".format(np.mean(scores)))

Accuracy score in each iteration: 
[0.9912280701754386, 0.9473684210526315, 0.9649122807017544, 0.9649122807017544, 0.9473684210526315]

K-Fold Score: 0.9631578947368421


### Using Scikit-learn Library:

    - cross_val_score: To compute score of each test fold. 
    - cross_val_predict: To predict and returns predicted score of each test fold.

Default, integer cv parameter assumes StratifiedK-Fold. As shown below, it is Stratified 5-folds implemented via cross_val_Score but you can also set cv to K-Fold commented.

In [44]:
model = LogisticRegression()

cv_scores_5_folds = cross_val_score(model,X,y,cv=5)
#cv_scores_5_folds = cross_val_score(model,X,y,cv=KFold(n_splits=5))

cv_predicts_5_folds = cross_val_predict(model,X,y,cv=5)

print("Accuracy score in each iteration: {} \n".format(cv_scores_5_folds))
print("Predicted class for each record: {} \n".format(cv_predicts_5_folds))
print("K-Fold Score: {}\n".format(np.mean(cv_scores_5_folds)))
print("Total records: {}, Total predicted values: {}".format(df.shape[0],len(cv_predicts_5_folds)))

Accuracy score in each iteration: [0.93859649 0.93859649 0.95614035 0.92105263 0.96460177] 

Predicted class for each record: [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 0 0 1 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 1
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 0 0 1 1 0 0 1 1 1 1 0 1 1 0 0 0 1 1
 1 0 1 1 1 0 1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 1 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1
 1 0 1 0 1 1 1 0 0 1 1 0 1 1 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0 1 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 0 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1 1 1 0 0 0 1 1
 1 1 0 1 0 1 0 1 1 1 0 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 0
 0 1 0 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 

## Cross-Validation

### Choose your model
    Aim: To compare classifier using cross_val_score. 
    Conclusion: Not much difference in scores but RandomForest performed best among three

In [39]:
rf_cv_scores_5_folds = cross_val_score(RandomForestClassifier(random_state=seed),X,y,cv=5)
print("K-Fold Score of RandomForest: {}".format(np.mean(rf_cv_scores_5_folds)))

dt_cv_scores_5_folds = cross_val_score(DecisionTreeClassifier(random_state=seed),X,y,cv=5)
print("K-Fold Score of Decision Tree: {}".format(np.mean(dt_cv_scores_5_folds)))

log_cv_scores_5_folds = cross_val_score(LogisticRegression(random_state=seed),X,y,cv=5)
print("K-Fold Score of Logistic Regression: {}".format(np.mean(log_cv_scores_5_folds)))

K-Fold Score of RandomForest: 0.9560937742586555
K-Fold Score of Decision Tree: 0.9208818506443098
K-Fold Score of Logistic Regression: 0.943797546964757


## Hyper-parameter Tuning of RandomForest 

- Model which was simply trained on manually estimated parameters may perform better if its parameters get more tuned as per data. This tuning is necessary to improvise your model which is about testing cartesian product of parameters using Grid Search or Random Search. 

- <b>GridSearchCV: </b>
Scikit-learn library to define ranges of parameters, perform CrossValidation with each combination of cartesian product of hyperparameters.

In [40]:
rf_model = RandomForestClassifier(random_state=seed)

In [41]:
n_estimators = list(range(5,20))
maxdepth = list(range(2,10)) 

hyperparas = dict(n_estimators=n_estimators,max_depth=maxdepth)
grid_rf_model = GridSearchCV(estimator=rf_model, param_grid=hyperparas, cv=5)
improved_model = grid_rf_model.fit(X,y)

In [42]:
print("Best score:",improved_model.best_score_)
print("Best parameters for best score:",improved_model.best_params_)

Best score: 0.9578326346840551
Best parameters for best score: {'max_depth': 9, 'n_estimators': 17}


### Testing best parameters returned by GridSearchCV
    Conclusion: Score is improved in an accuracy of approx 1%

In [43]:
rf_model = RandomForestClassifier(max_depth=6, n_estimators= 19, random_state=seed)
score = cross_val_score(rf_model,X,y,cv=5)
print("CV Score:",np.mean(score))

CV Score: 0.9525850023288308
